In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import numpy as np
import hvplot.pandas
from pathlib import Path
from sklearn.linear_model import LinearRegression
import tensorflow as tf

2024-02-22 19:18:13.840743: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Import our input dataset
salary_df = pd.read_csv('ds_salaries.csv')
salary_df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
3752,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


In [3]:
salary_df.drop(salary_df[["salary", "salary_currency"]], axis = 1, inplace=True)
salary_df.head()

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,CA,100,CA,M


In [4]:
 # Check the number of unique values in each column
salary_df.nunique()

work_year                4
experience_level         4
employment_type          4
job_title               93
salary_in_usd         1035
employee_residence      78
remote_ratio             3
company_location        72
company_size             3
dtype: int64

In [5]:
salary_df["job_title"].value_counts()

Data Engineer                          1040
Data Scientist                          840
Data Analyst                            612
Machine Learning Engineer               289
Analytics Engineer                      103
                                       ... 
Principal Machine Learning Engineer       1
Azure Data Engineer                       1
Manager Data Management                   1
Marketing Data Engineer                   1
Finance Data Analyst                      1
Name: job_title, Length: 93, dtype: int64

In [6]:
engineer = '.*Engineer'
salary_df[salary_df['job_title'].str.contains(engineer)]

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
1,2023,MI,CT,ML Engineer,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,US,100,US,S
21,2023,SE,FT,Research Engineer,275000,DE,0,DE,M
22,2023,SE,FT,Research Engineer,174000,DE,0,DE,M
23,2023,SE,FT,Analytics Engineer,230000,GB,100,GB,M
...,...,...,...,...,...,...,...,...,...
3723,2021,SE,FT,Computer Vision Engineer,18907,BR,0,BR,M
3739,2021,MI,FT,Data Engineer,45391,NL,100,NL,L
3743,2020,MI,FT,Data Engineer,130800,ES,100,US,M
3744,2020,SE,FT,Machine Learning Engineer,45618,HR,100,HR,S


In [7]:
analyst = '.*Analyst'
salary_df[salary_df['job_title'].str.contains(analyst)]

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
11,2023,SE,FT,Data Analyst,130000,US,100,US,M
12,2023,SE,FT,Data Analyst,100000,US,100,US,M
19,2023,MI,FT,Data Analyst,150000,US,100,US,M
20,2023,MI,FT,Data Analyst,110000,US,100,US,M
37,2023,MI,FT,Data Analyst,105380,US,0,US,M
...,...,...,...,...,...,...,...,...,...
3724,2021,EN,FT,Business Data Analyst,59102,LU,100,LU,L
3734,2021,MI,FT,Lead Data Analyst,19609,IN,100,IN,L
3736,2021,MI,FT,Data Analyst,75000,US,0,US,L
3737,2021,MI,FT,Data Analyst,62000,US,0,US,L


In [8]:
scientist = '.*Scientist'
salary_df[salary_df['job_title'].str.contains(scientist)]

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,85847,ES,100,ES,L
3,2023,SE,FT,Data Scientist,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,CA,100,CA,M
5,2023,SE,FT,Applied Scientist,222200,US,0,US,L
6,2023,SE,FT,Applied Scientist,136000,US,0,US,L
...,...,...,...,...,...,...,...,...,...
3746,2021,MI,FT,Data Scientist,119059,SG,100,IL,M
3747,2021,MI,FT,Applied Machine Learning Scientist,423000,US,50,US,L
3750,2020,SE,FT,Data Scientist,412000,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,US,100,US,L


In [9]:
engineer = 'Engineer'
analyst = 'Analyst'
scientist = 'Scientist'

for idx, x in enumerate(salary_df["job_title"]):
    if engineer in x:
        salary_df.at[idx, "job_title"] = "Data Engineer"
    elif analyst in x:
        salary_df.at[idx, "job_title"] = "Data Analyst"
    elif scientist in x:
        salary_df.at[idx, "job_title"] = "Data Scientist"
    else:
        salary_df.at[idx, "job_title"] = "Other"
salary_df["job_title"]

0       Data Scientist
1        Data Engineer
2        Data Engineer
3       Data Scientist
4       Data Scientist
             ...      
3750    Data Scientist
3751    Data Scientist
3752    Data Scientist
3753      Data Analyst
3754             Other
Name: job_title, Length: 3755, dtype: object

In [10]:
salary_df["job_title"].value_counts()

Data Engineer     1640
Data Scientist    1065
Data Analyst       684
Other              366
Name: job_title, dtype: int64

In [11]:
salary_df["employee_residence"].value_counts()

US    3004
GB     167
CA      85
ES      80
IN      71
      ... 
BA       1
AM       1
CY       1
KW       1
MT       1
Name: employee_residence, Length: 78, dtype: int64

In [12]:
US = "US"
for idx, x in enumerate(salary_df["employee_residence"]):
    if US in x:
        salary_df.at[idx, "employee_residence"] = "US"
    else:
        salary_df.at[idx, "employee_residence"] = "Other"
salary_df["employee_residence"].head()

0    Other
1       US
2       US
3    Other
4    Other
Name: employee_residence, dtype: object

In [13]:
salary_df["employee_residence"].value_counts()

US       3004
Other     751
Name: employee_residence, dtype: int64

In [14]:
salary_df["company_location"].value_counts()

US    3040
GB     172
CA      87
ES      77
IN      58
      ... 
MK       1
BS       1
IR       1
CR       1
MT       1
Name: company_location, Length: 72, dtype: int64

In [15]:
for idx, x in enumerate(salary_df["company_location"]):
    if US in x:
        salary_df.at[idx, "company_location"] = "US"
    else:
        salary_df.at[idx, "company_location"] = "Other"
salary_df["company_location"].head()

0    Other
1       US
2       US
3    Other
4    Other
Name: company_location, dtype: object

In [16]:
salary_df["company_location"].value_counts()

US       3040
Other     715
Name: company_location, dtype: int64

In [17]:
salary_df.dtypes

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

In [18]:
salary_df["work_year"] = salary_df["work_year"].astype(str)
salary_df["remote_ratio"] = salary_df["remote_ratio"].astype(str)

salary_df.dtypes

work_year             object
experience_level      object
employment_type       object
job_title             object
salary_in_usd          int64
employee_residence    object
remote_ratio          object
company_location      object
company_size          object
dtype: object

In [19]:
salary_df = salary_df[(salary_df['employee_residence'] == 'US')]
salary_df.head(10)

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
1,2023,MI,CT,Data Engineer,30000,US,100,US,S
2,2023,MI,CT,Data Engineer,25500,US,100,US,S
5,2023,SE,FT,Data Scientist,222200,US,0,US,L
6,2023,SE,FT,Data Scientist,136000,US,0,US,L
9,2023,SE,FT,Data Scientist,147100,US,0,US,M
10,2023,SE,FT,Data Scientist,90700,US,0,US,M
11,2023,SE,FT,Data Analyst,130000,US,100,US,M
12,2023,SE,FT,Data Analyst,100000,US,100,US,M
13,2023,EN,FT,Data Scientist,213660,US,0,US,L
14,2023,EN,FT,Data Scientist,130760,US,0,US,L


In [20]:
#Build Nueral Network

salary_with_dummies = pd.get_dummies(salary_df, columns=['work_year', 'experience_level', 'employment_type', 'job_title', 'employee_residence', 'remote_ratio', 'company_location', 'company_size'], dtype=int)

print(salary_with_dummies.head())
salary_with_dummies.dtypes

   salary_in_usd  work_year_2020  work_year_2021  work_year_2022  \
1          30000               0               0               0   
2          25500               0               0               0   
5         222200               0               0               0   
6         136000               0               0               0   
9         147100               0               0               0   

   work_year_2023  experience_level_EN  experience_level_EX  \
1               1                    0                    0   
2               1                    0                    0   
5               1                    0                    0   
6               1                    0                    0   
9               1                    0                    0   

   experience_level_MI  experience_level_SE  employment_type_CT  ...  \
1                    1                    0                   1  ...   
2                    1                    0                   1  ...

salary_in_usd               int64
work_year_2020              int64
work_year_2021              int64
work_year_2022              int64
work_year_2023              int64
experience_level_EN         int64
experience_level_EX         int64
experience_level_MI         int64
experience_level_SE         int64
employment_type_CT          int64
employment_type_FT          int64
employment_type_PT          int64
job_title_Data Analyst      int64
job_title_Data Engineer     int64
job_title_Data Scientist    int64
job_title_Other             int64
employee_residence_US       int64
remote_ratio_0              int64
remote_ratio_100            int64
remote_ratio_50             int64
company_location_Other      int64
company_location_US         int64
company_size_L              int64
company_size_M              int64
company_size_S              int64
dtype: object

In [21]:
# Split our preprocessed data into our features and target arrays
y = salary_with_dummies['salary_in_usd'].values
X = salary_with_dummies.drop(columns=['salary_in_usd'])


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
X.dtypes


work_year_2020              int64
work_year_2021              int64
work_year_2022              int64
work_year_2023              int64
experience_level_EN         int64
experience_level_EX         int64
experience_level_MI         int64
experience_level_SE         int64
employment_type_CT          int64
employment_type_FT          int64
employment_type_PT          int64
job_title_Data Analyst      int64
job_title_Data Engineer     int64
job_title_Data Scientist    int64
job_title_Other             int64
employee_residence_US       int64
remote_ratio_0              int64
remote_ratio_100            int64
remote_ratio_50             int64
company_location_Other      int64
company_location_US         int64
company_size_L              int64
company_size_M              int64
company_size_S              int64
dtype: object

In [23]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Number of input features
input_features = len(X_train_scaled[0])

# Define the model - deep neural net
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=16, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=10, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                400       
                                                                 
 dense_1 (Dense)             (None, 10)                170       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 581 (2.27 KB)
Trainable params: 581 (2.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
input_features

24

In [26]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
import numpy as np
import tensorflow as tf

# Convert X_train and y_train to NumPy arrays or TensorFlow tensors
X_train = np.array(X_train)
y_train = np.array(y_train)

In [28]:
from tensorflow.keras.callbacks import ModelCheckpoint
# Create a callback to save the model's weights every five epochs
checkpoint_callback = ModelCheckpoint(filepath='model_weights.{epoch:02d}.hdf5', save_freq='epoch', period=5)

# Train the model
history = nn.fit(X_train, y_train, epochs=25, validation_data=(X_test, y_test))

Epoch 1/25
76/76 [==============================] - 1s 3ms/step - loss: -347867.1875 - accuracy: 0.0000e+00 - val_loss: -870036.0000 - val_accuracy: 0.0000e+00
Epoch 2/25
76/76 [==============================] - 0s 1ms/step - loss: -2301381.5000 - accuracy: 0.0000e+00 - val_loss: -4653512.0000 - val_accuracy: 0.0000e+00
Epoch 3/25
76/76 [==============================] - 0s 1ms/step - loss: -8924784.0000 - accuracy: 0.0000e+00 - val_loss: -14794103.0000 - val_accuracy: 0.0000e+00
Epoch 4/25
76/76 [==============================] - 0s 1ms/step - loss: -23173090.0000 - accuracy: 0.0000e+00 - val_loss: -34163996.0000 - val_accuracy: 0.0000e+00
Epoch 5/25
76/76 [==============================] - 0s 1ms/step - loss: -48252760.0000 - accuracy: 0.0000e+00 - val_loss: -66017964.0000 - val_accuracy: 0.0000e+00
Epoch 6/25
76/76 [==============================] - 0s 1ms/step - loss: -87109840.0000 - accuracy: 0.0000e+00 - val_loss: -113674912.0000 - val_accuracy: 0.0000e+00
Epoch 7/25
76/76 [====

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

19/19 - 0s - loss: -6.4474e+09 - accuracy: 0.0000e+00 - 27ms/epoch - 1ms/step
Loss: -6447356928.0, Accuracy: 0.0
